In [4]:
import os
from mistralai import Mistral

api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-large-2411"

client = Mistral(
    api_key=api_key,
)

# Check API is alive
chat_response = client.chat.complete(
    model=model,
    messages=[
        {
            "role": "user",
            "content": "What is the best French cheese?",
        },
    ],
    max_tokens=10,
)
print(chat_response.model, chat_response.choices[0].message.content)

mistral-large-2411 Determining the "best" French cheese can


In [5]:
from openai import RateLimitError
from mistralai import SDKError
from time import sleep


def wait():
    sleep(1.5)


def repeat_if_hit_api_limit(f):  # (1)
    def wrapper(*args, **kw):  # (2)
        while True:
            try:
                return f(*args, **kw)
            except RateLimitError:
                wait()
            except SDKError as e:
                if e.status_code == 429:
                    wait()
                else:
                    raise e

    return wrapper

In [ ]:
import os, sys

dir2 = os.path.abspath("")
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)

import pandas as pd
import ast
import csv
from tqdm import tqdm
import re

import utils.prompt as prompt

import importlib

# Required to purge the module cache and use the latest version after an update
importlib.reload(prompt)

difficulty = ["middle_school", "high_school", "undergraduate", "postgraduate", "phd"]
ratings = list(range(1, 11, 1))

invalid_complexities = 0
invalid_ratings = 0


@repeat_if_hit_api_limit
def model_as_judge(client, model, index, system_prompt, user_prompt, answer):
    global invalid_ratings

    chat_response = client.chat.complete(
        model=model,
        messages=[
            {
                "role": "system",
                "content": 'Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user request displayed below. Your evaluation should consider factors such as the following all the settings in the system prompt, correspondences to the context of the user, the helpfulness, relevance and accuracy. Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must rate the response on a scale of 1 to 10 by strictly following this format: "[[rating]]", for example:"Rating: [[6]]".',
            },
            {
                "role": "user",
                "content": f"""
                [Instructions for Assistant]
                {system_prompt}
                [End of Instructions for Assistant]

                [Question]
                {user_prompt}
                [End of Question]

                [The Start of Assistant’s Answer]
                {answer}
                [The End of Assistant’s Answer]
                """,
            },
        ],
    )
    response = chat_response.choices[0].message.content
    # print(response)

    try:
        rating = re.search("\\[\\[(\\d+?)\\]\\]", response).group(1)
        # print(rating)
        rating_int = int(rating)
        if rating_int in ratings:
            df.at[index, "masj_rating"] = rating_int
        else:
            invalid_ratings += 1
    except:
        print(f"Could not extract rating from response:\n{response}\n")
        invalid_ratings += 1


@repeat_if_hit_api_limit
def estimate_complextiy_with_model(client, model, index, system_prompt, user_prompt):
    global invalid_complexities

    chat_response = client.chat.complete(
        model=model,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": f"""
                [Question Start]
                {user_prompt}
                [Question End]
                """,
            },
        ],
    )
    response = chat_response.choices[0].message.content
    # print(response)

    try:
        complexity = re.search("\\[\\[(.+?)\\]\\]", response).group(1)
        # print(complexity)

        if complexity in difficulty:
            df.at[index, "masj_complexity"] = complexity
        else:
            invalid_complexities += 1
    except:
        print(f"Could not extract complexity from response:\n{response}\n")
        invalid_complexities += 1

    return response


DUMP_EVERY = 100


def estimate_dataset(df, client, get_question_from_row, get_options_from_row, out_filename):
    if "masj_complexity" not in df.columns:
        df["masj_complexity"] = ""
    if "masj_rating" not in df.columns:
        df["masj_rating"] = 0

    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        if df.at[index, "masj_complexity"] in difficulty and df.at[index, "masj_rating"] in ratings:
            continue

        complexity_system_prompt = f'You are an expert in the topic of the question. Please act as an impartial judge and evaluate the complexity of the multiple-choice question with options below. Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must not answer the question. You must rate the question complexity by strictly following the scale: {", ".join(difficulty)}. You must return the complexity by strictly following this format: "[[complexity]]", for example: "Complexity: [[middle_school]]".'
        complexity_user_prompt = prompt.get_user_prompt(get_question_from_row(row), get_options_from_row(row))

        response_complexity = estimate_complextiy_with_model(
            client, model, index, complexity_system_prompt, complexity_user_prompt
        )
        wait()

        model_as_judge(client, model, index, complexity_system_prompt, complexity_user_prompt, response_complexity)
        wait()

        if index % DUMP_EVERY == 0:
            df.to_csv(out_filename, sep="\t", quoting=csv.QUOTE_NONE, quotechar="", escapechar="\\", index=False)

    df.to_csv(out_filename, sep="\t", quoting=csv.QUOTE_NONE, quotechar="", escapechar="\\", index=False)
    print(
        f"Processed dataset {out_filename}. Total entries: {df.shape[0]}. Invalid complexities: {invalid_complexities}. Invalid ratings: {invalid_ratings}"
    )
    return df


DATASET = "../data/mmlu_pro_stem"
out_filename = f"{DATASET}_w_maj_complexity.tsv"

df = pd.read_csv(out_filename, sep="\t", header=0, quoting=csv.QUOTE_NONE, quotechar="", escapechar="\\")
# df = df.head(10)


estimate_dataset(
    df=df,
    client=client,
    get_question_from_row=lambda row: row["question"],
    get_options_from_row=lambda row: ast.literal_eval(row["options"]),
    out_filename=out_filename,
)

  8%|▊         | 971/12032 [11:52<26:42:57,  8.70s/it]

Could not extract rating from response:
The assistant's response demonstrates a clear understanding of the economic concepts involved in the question. The explanation highlights that the question requires not just knowledge of terms like monopolistic competition and pure competition, but also the ability to apply these concepts to a hypothetical scenario and consider the market dynamics. This level of analysis is typically expected at the undergraduate level in economics.

Complexity: [[undergraduate]]



  8%|▊         | 972/12032 [11:59<25:41:36,  8.36s/it]

Could not extract rating from response:
The assistant's response provides a clear and concise explanation of the context and requirements of the question. It accurately identifies that the question relates to economics and business studies, specifically the concept of production efficiency. The response highlights that answering the question correctly requires an understanding of different efficiency concepts, which is typically covered at the undergraduate level in economics or business courses. This evaluation is objective and relevant to the context of the question.

Complexity: [[undergraduate]]



  9%|▊         | 1036/12032 [21:05<27:40:18,  9.06s/it]

...

 13%|█▎        | 1621/12032 [1:46:42<23:35:33,  8.16s/it]

Could not extract rating from response:
The explanation provided by the assistant is clear and concise, accurately describing the mathematical concepts required to solve the problem. The question involves understanding percentage increases and performing basic arithmetic, which are typically covered in middle school mathematics curricula. The assistant correctly identifies that the student needs to calculate a 1% increase on the current tax amount, which involves multiplying by 1.01. This evaluation is relevant, accurate, and helpful in understanding the complexity of the question.

Complexity: [[middle_school]]



 14%|█▍        | 1679/12032 [1:54:40<20:36:56,  7.17s/it]

Could not extract rating from response:
The assistant's response provides a clear and concise explanation of the knowledge required to answer the question. The question indeed requires an understanding of tax structures and the concept of regressive taxes, which are typically introduced in high school economics or social studies curricula. The question does not require advanced mathematical calculations or deep theoretical knowledge, making it accessible to a high school student with a basic understanding of economics.

Complexity: [[high_school]]



 14%|█▍        | 1702/12032 [1:57:52<22:12:22,  7.74s/it]